# Code generation for recommender workflows

Anton Antonov  
RakuForPrediction at WordPress   
October 2025

----

## Setup

In [26]:
use Data::Reshapers;
use Data::Importers;
use Data::Summarizers;

use DSL::Translators;
use DSL::Examples;
use ML::NLPTemplateEngine;

use Math::SparseMatrix :ALL;
use Math::SparseMatrix::DOK;
use Math::SparseMatrix::Utilities;

use ML::SparseMatrixRecommender;

----

## Ingestion

In [2]:
my $url = 'https://raw.githubusercontent.com/antononcube/MathematicaVsR/refs/heads/master/Data/MathematicaVsR-Data-Titanic.csv';
my @dsData = data-import($url, headers => 'auto');
#@dsData .= map({ $_<id> = 'id.' ~ $_<id>; $_ });
#@dsData .= map({ $_<passengerAge> = $_<passengerAge>.Int; $_ });
@dsData.&dimensions

(1309 5)

In [3]:
deduce-type(@dsData);

Vector(Assoc(Atom((Str)), Atom((Str)), 5), 1309)

---

## SMR

In [20]:
my $smrObj = 
    ML::SparseMatrixRecommender.new
    .create-from-wide-form(@dsData,
        item-column-name => "id",
        tag-types => <passengerSex passengerClass passengerAge passengerSurvival>,
        :add-tag-types-to-column-names,
        tag-value-separator => ":")
    .apply-term-weight-functions("IDF", "None", "Cosine")

ML::SparseMatrixRecommender(:matrix-dimensions((1309, 17)), :density(<4/17>), :tag-types(("passengerClass", "passengerSex", "passengerAge", "passengerSurvival")))

In [5]:
$smrObj.take-M.column-names

[passengerSex:female passengerSex:male passengerClass:1st passengerClass:2nd passengerClass:3rd passengerSurvival:died passengerSurvival:survived]

In [6]:
#% html
my @field-names = 'score', 'id', |@dsData.head.keys.grep(* ne 'id').sort;
$smrObj
.recommend-by-profile({"passengerSex:male" => 1, "passengerAge:30" => 20}, 12)
.join-across(@dsData)
.take-value
==> to-html(:@field-names)   

score,id,passengerAge,passengerClass,passengerSex,passengerSurvival
1,10,70,1st,male,died
1,1001,-1,3rd,male,survived
1,1004,-1,3rd,male,survived
1,1006,-1,3rd,male,died
1,101,50,1st,male,survived
1,1010,-1,3rd,male,died
1,1011,20,3rd,male,died
1,1016,60,3rd,male,died
1,1017,-1,3rd,male,died
1,1018,20,3rd,male,survived


In [22]:
$smrObj.take-matrices.keys

(passengerClass passengerSex passengerAge passengerSurvival)

In [24]:
#% html
my @field-names = [|<score id>, |$smrObj.take-matrices.keys.sort];
sink $smrObj
.recommend(<1089 1132>, 10, :!remove-history)
.echo-value
.join-across(@dsData, on => 'id')
.echo-value(as => {&to-pretty-table($_, :@field-names)});

[1089 => 8 1132 => 8 1197 => 8 1229 => 8 614 => 8 631 => 8 675 => 8 715 => 8 747 => 8 761 => 8]
+----------+------+--------------+----------------+--------------+-------------------+
|  score   |  id  | passengerAge | passengerClass | passengerSex | passengerSurvival |
+----------+------+--------------+----------------+--------------+-------------------+
| 8.000000 | 1089 |      30      |      3rd       |     male     |      survived     |
| 8.000000 | 1132 |      30      |      3rd       |     male     |      survived     |
| 8.000000 | 1197 |      30      |      3rd       |     male     |      survived     |
| 8.000000 | 1229 |      30      |      3rd       |     male     |      survived     |
| 8.000000 | 614  |      30      |      3rd       |     male     |      survived     |
| 8.000000 | 631  |      30      |      3rd       |     male     |      survived     |
| 8.000000 | 675  |      30      |      3rd       |     male     |      survived     |
| 8.000000 | 715  |      30      |

----

## DSL translation (grammars, ProdGDT)

In [ ]:
'
create from @dsData; 
apply LSI functions IDF, None, Cosine; 
recommend by profile for passengerSex:male, and passengerClass:1st
'
==> dsl-web-translation(format => 'CODE')

In [ ]:
'
create from @dsData; 
apply LSI functions IDF, None, Cosine; 
recommend by profile for passengerSex:male, and passengerClass:1st;
join across with @dsData on "id";
echo the pipeline value
'
==> ToDSLCode(to => 'Raku', format => 'CODE')

----

## DSL Translation (LLM examples)

In [ ]:
#% html
dsl-examples().map({ $_.key X $_.value.keys }).flat(1).map({ <language workflow> Z=> $_ })».Hash.sort.Array
==> to-dataset()
==> to-html(field-names => <language workflow>)

In [ ]:
my &llm-pipeline-segment = llm-example-function(dsl-examples()<Raku><SMRMon>);

In [ ]:
my $spec = q:to/END/;
new recommender;
create from @dsData; 
apply LSI functions IDF, None, Cosine; 
recommend by profile for passengerSex:male, and passengerClass:1st;
join across with @dsData on "id";
echo the pipeline value;
classify by profile passengerSex:female, and passengerClass:1st on the tag passengerSurvival;
echo value
END

sink my @commands = $spec.lines;

In [ ]:
@commands
.map({ .&llm-pipeline-segment })
.map({ .subst(/:i Output ':'?/):g })
.join("\n.")

----

## NLP Template Engine

In [31]:
'create with dfTitanic; apply the LSI functions IDF, None, Cosine;recommend by profile 1st and male'
==> concretize(lang => "Raku")


my $smrObj = ML::SparseMatrixRecommender.new
.create-from-wide-form(dfTitanic, item-column-name='id', :add-tag-types-to-column-names, tag-value-separator=':')
.apply-term-weight-functions('IDF', 'None', 'Cosine')
.recommend-by-profile(["1st"], 12, :!normalize)
.join-across(dfTitanic)
.echo-value();